<a href="https://colab.research.google.com/github/sreent/machine-learning/blob/main/Final%20DNN%20Code%20Examples/Twitter%20Entity%20Sentiment/Twitter%20Entity%20Sentiment%20-%20NLP%20Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitter Entity Sentiment - NLP Example

This notebook demonstrates the **Universal ML Workflow** applied to a multi-class NLP classification problem using Twitter entity sentiment data.

## Learning Objectives

By the end of this notebook, you will be able to:
- Apply the Universal ML Workflow to an NLP text classification problem
- Convert text data to numerical features using **TF-IDF (Term Frequency-Inverse Document Frequency)** vectorisation
- Handle **imbalanced classes** using class weights during training
- Build and train deep neural networks for **multi-class classification** (4 classes)
- Use **Hyperband** for efficient hyperparameter tuning
- Apply **Dropout + L2 regularisation** to prevent overfitting
- Evaluate model performance using appropriate metrics for imbalanced data (**F1-Score**, Accuracy, Precision, Recall)

---

## Dataset Overview

| Attribute | Description |
|-----------|-------------|
| **Source** | [Kaggle Twitter Entity Sentiment Analysis](https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis) |
| **Problem Type** | Multi-Class Classification (4 classes) |
| **Classes** | Irrelevant, Negative, Neutral, Positive |
| **Data Balance** | Imbalanced |
| **Data Type** | Unstructured Text (Tweets) |
| **Input Features** | TF-IDF Vectors (5000 features, bigrams) |
| **Output** | Sentiment: Irrelevant, Negative, Neutral, or Positive |
| **Imbalance Handling** | Class Weights during Training |

---

## Technique Scope

This notebook uses only techniques from **Chapters 1–4** of *Deep Learning with Python* (Chollet, 2021). This means:

| Technique | Status | Rationale |
|-----------|--------|----------|
| **Dense layers (MLP/DNN)** | ✓ Used | Core building block (Ch. 3-4) |
| **Dropout** | ✓ Used | Regularisation technique (Ch. 4) |
| **L2 regularisation** | ✓ Used | Weight penalty (Ch. 4) |
| **Early stopping** | ✗ Not used | Introduced in Ch. 7 |
| **CNN** | ✗ Not used | Introduced in Ch. 8 |
| **RNN/LSTM** | ✗ Not used | Introduced in Ch. 10 |

We demonstrate that **Dropout + L2 regularisation** alone can effectively prevent overfitting without requiring early stopping.

---

## 1. Defining the Problem and Assembling a Dataset

The first step in any machine learning project is to clearly define the problem and understand the data.

**Problem Statement:** Given a tweet mentioning an entity (brand, product, person), predict the sentiment expressed (Irrelevant, Negative, Neutral, or Positive).

**Why this matters:** Entity-level sentiment analysis helps:
- Brands monitor public perception of their products
- Marketers understand customer reactions to campaigns
- Analysts track sentiment trends over time

**Key difference from 3-class sentiment:** This dataset includes an **Irrelevant** class for tweets that mention the entity but don't express sentiment about it. This is common in real-world applications where not all mentions are relevant.

**Data Source:** This dataset contains tweets with entity-level sentiment annotations from Kaggle.

## 2. Choosing a Measure of Success

### Metric Selection Based on Class Imbalance

The choice of evaluation metric depends on **class imbalance**. We use practical guidelines derived from the literature:

| Imbalance Ratio | Classification | Primary Metric | Rationale |
|-----------------|----------------|----------------|----------|
| ≤ 1.5:1 | Balanced | **Accuracy** | Classes roughly equal |
| 1.5:1 – 3:1 | Mild Imbalance | **Accuracy** | Majority class < 75% |
| > 3:1 | Moderate/Severe | **F1-Score** | Accuracy becomes misleading |

**Why these thresholds?**
- **3:1 ratio**: When majority class exceeds 75%, a naive classifier achieves high accuracy while ignoring minority classes
- **F1-Score**: Harmonic mean of precision and recall, effective for imbalanced data (He and Garcia, 2009)

### References

- Branco, P., Torgo, L. and Ribeiro, R.P. (2016) 'A survey of predictive modelling on imbalanced domains', *ACM Computing Surveys*, 49(2), pp. 1–50.

- He, H. and Garcia, E.A. (2009) 'Learning from imbalanced data', *IEEE Transactions on Knowledge and Data Engineering*, 21(9), pp. 1263–1284.

*Note: The 3:1 threshold is a practical guideline, not a strict academic standard. The literature suggests metric choice depends on domain-specific costs of errors.*

## 3. Deciding on an Evaluation Protocol

### Hold-Out vs K-Fold Cross-Validation

The choice between hold-out validation and k-fold cross-validation is a trade-off between estimate stability and computational cost. For model selection, k-fold cross-validation is widely used because it averages performance over multiple splits; Kohavi (1995) reports stratified 10-fold cross-validation as a strong general default for model selection. However, k-fold cross-validation requires training the model k times, which can be computationally expensive, especially for larger datasets and heavier models.

| Situation        | Recommended method                                   | Rationale |
|-----------------|--------------------------------------------------------|-----------|
| Smaller datasets | Stratified k-fold cross-validation (commonly 5 or 10 folds) | A single hold-out split may be unstable when the validation set is small; averaging across folds typically provides a more reliable model-selection signal. |
| Larger datasets  | Hold-out validation + separate test set                | With sufficient data, a single validation split is often adequate while avoiding the k× training cost of k-fold; a held-out test set supports final unbiased reporting. |

*Note:* This table summarises a rule-of-thumb stability–cost trade-off rather than fixed numeric cut-offs.


### References

- Chollet, F. (2021) *Deep learning with Python*. 2nd edn. Shelter Island, NY: Manning Publications.

- Hastie, T., Tibshirani, R. and Friedman, J. (2009) *The elements of statistical learning: data mining, inference, and prediction*. 2nd edn. New York: Springer.

- Kohavi, R. (1995) 'A study of cross-validation and bootstrap for accuracy estimation and model selection', *Proceedings of the 14th International Joint Conference on Artificial Intelligence*, 2, pp. 1137–1145.

- Pedregosa, F. et al. (2011) 'Scikit-learn: machine learning in Python', *Journal of Machine Learning Research*, 12, pp. 2825–2830. Available at: https://scikit-learn.org/stable/modules/cross_validation.html (Accessed: 20 January 2025).

*Note: The 10,000 threshold is a practical guideline. For computationally cheap models, K-fold is preferred regardless of size.*

### Data Split Strategy (This Notebook)

```
Original Data → Hold-Out Selected
├── Test Set (10%) - Final evaluation only
└── Training Pool (90%)
    ├── Training Set (81%) - Model training
    └── Validation Set (9%) - Hyperparameter tuning
```

**Important:** We use `stratify` parameter to maintain class proportions in all splits.

## 4. Preparing Your Data

### 4.1 Import Libraries and Set Random Seed

We set random seeds for reproducibility - this ensures that running the notebook multiple times produces the same results.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Keras Tuner for hyperparameter search
%pip install -q -U keras-tuner
import keras_tuner as kt

import matplotlib.pyplot as plt

SEED = 204

tf.random.set_seed(SEED)
np.random.seed(SEED)

import warnings
warnings.filterwarnings('ignore')

### 4.2 Load and Explore the Dataset

Let's download the Twitter entity sentiment data from Google Drive and examine its structure.

In [ ]:
# Load data directly from Google Drive
GDRIVE_FILE_ID = '14RgPMuGdicl0lg10L7T8tFEnYs9PXudK'
DATA_URL = f'https://drive.google.com/uc?id={GDRIVE_FILE_ID}&export=download'

# This dataset has no header row
tweets = pd.read_csv(DATA_URL, header=None)

# Columns: 0=tweet_id, 1=entity, 2=sentiment, 3=text
# We only need sentiment and text
tweets = tweets[[2, 3]]
tweets.columns = ['sentiment', 'text']

# Remove any rows with missing values
tweets.dropna(inplace=True)

tweets.head()

### 4.3 Split Data into Train and Test Sets

We reserve 10% of the data for final testing. The `stratify` parameter ensures that each split maintains the same class proportions as the original dataset - critical for imbalanced data.

In [ ]:
TEST_SIZE = 0.1

(tweets_train, tweets_test,
 sentiment_train, sentiment_test) = train_test_split(tweets['text'], tweets['sentiment'],
                                                     test_size=TEST_SIZE, stratify=tweets['sentiment'],
                                                     shuffle=True, random_state=SEED)

### 4.4 Text Vectorisation with TF-IDF

Neural networks require numerical input, but tweets are text. We use **TF-IDF (Term Frequency-Inverse Document Frequency)** to convert text to numbers.

**How TF-IDF works:**
- **TF (Term Frequency):** How often a word appears in a document
- **IDF (Inverse Document Frequency):** Downweights words that appear in many documents (like "the", "is")
- **TF-IDF = TF × IDF:** Words that are frequent in a document but rare overall get high scores

**Our settings:**
- `max_features=5000`: Keep only the 5000 most important terms
- `ngram_range=(1, 2)`: Include both single words (unigrams) and word pairs (bigrams) like "great service"

---

#### Design Decisions

**Why TF-IDF instead of Word Embeddings (Word2Vec, GloVe)?**

| Approach | Pros | Cons |
|----------|------|------|
| **TF-IDF** | Simple, interpretable, no pretrained models needed | Loses word order, no semantic similarity |
| **Word Embeddings** | Captures semantic meaning, dense vectors | Requires pretrained models or training, more complex |

We use TF-IDF because:
1. **Technique scope:** Word embeddings are introduced in Chapter 11 of *Deep Learning with Python* (Chollet, 2021), outside our Ch. 1-4 scope
2. **Simplicity:** TF-IDF works well for sentiment classification where specific words (e.g., "terrible", "amazing") are strong predictors
3. **Interpretability:** Feature weights directly correspond to terms

**Why 5000 features?**

This balances vocabulary coverage against dimensionality:
- Too few (e.g., 1000): May miss important domain-specific terms
- Too many (e.g., 20000): Increases noise and computational cost
- 5000 is a common practical choice that captures most meaningful terms while keeping the model tractable

**Why bigrams `(1, 2)` instead of just unigrams or trigrams?**

- **Unigrams only:** Misses phrases like "not good" or "very bad" where meaning depends on word pairs
- **Bigrams:** Captures common sentiment phrases ("great service", "long wait", "never again")
- **Trigrams:** Rarely add value for short texts like tweets; increases feature space significantly

**Why minimal text preprocessing?**

We don't apply stemming, lemmatization, or stopword removal because:
1. **TF-IDF handles common words:** IDF naturally downweights frequent words like "the", "is"
2. **Sentiment in stopwords:** Words like "not", "very", "but" carry sentiment information
3. **Simplicity:** Minimal preprocessing reduces pipeline complexity and potential errors
4. **Empirical evidence:** For sentiment analysis, heavy preprocessing often doesn't improve results (Haddi et al., 2013)

*Note: For other NLP tasks (e.g., topic modelling), preprocessing may be more beneficial.*

In [ ]:
MAX_FEATURES = 5000
NGRAMS = 2

tfidf = TfidfVectorizer(ngram_range=(1, NGRAMS), max_features=MAX_FEATURES)
tfidf.fit(tweets_train)

X_train, X_test = tfidf.transform(tweets_train).toarray(), tfidf.transform(tweets_test).toarray()

### 4.5 Encode Labels as One-Hot Vectors

For multi-class classification with softmax output, we need to convert categorical labels to one-hot encoded vectors:
- Irrelevant → [1, 0, 0, 0]
- Negative → [0, 1, 0, 0]
- Neutral → [0, 0, 1, 0]
- Positive → [0, 0, 0, 1]

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(tweets['sentiment'])

y_train = to_categorical(label_encoder.transform(sentiment_train))
y_test = to_categorical(label_encoder.transform(sentiment_test))

## 5. Developing a Model That Does Better Than a Baseline

Before building complex models, we need to establish **baseline performance**. This gives us a reference point to know if our model is actually learning something useful.

### 5.1 Examine Class Distribution

Let's look at how the sentiment classes are distributed:

In [ ]:
counts = tweets.groupby(['sentiment']).count()
counts.reset_index(inplace=True)

counts

In [ ]:
# =============================================================================
# DATA-DRIVEN ANALYSIS: Dataset Size & Imbalance
# =============================================================================

# Dataset size analysis (for hold-out vs K-fold decision)
n_samples = len(tweets)
HOLDOUT_THRESHOLD = 10000  # Use hold-out if samples > 10,000 (Kohavi, 1995; Chollet, 2021)

# Imbalance analysis (for metric selection)
majority_class = counts['text'].max()
minority_class = counts['text'].min()
imbalance_ratio = majority_class / minority_class
IMBALANCE_THRESHOLD = 3.0  # Use F1-Score if ratio > 3.0 (He & Garcia, 2009)

# Determine evaluation strategy and metric
use_holdout = n_samples > HOLDOUT_THRESHOLD
use_f1 = imbalance_ratio > IMBALANCE_THRESHOLD

print("=" * 60)
print("DATA-DRIVEN CONFIGURATION")
print("=" * 60)
print(f"\n1. DATASET SIZE: {n_samples:,} samples")
print(f"   Threshold: {HOLDOUT_THRESHOLD:,} samples (Kohavi, 1995)")
print(f"   Decision: {'Hold-Out' if use_holdout else 'K-Fold Cross-Validation'}")

print(f"\n2. CLASS IMBALANCE: {imbalance_ratio:.2f}:1 ratio")
print(f"   Threshold: {IMBALANCE_THRESHOLD:.1f}:1 (He & Garcia, 2009)")
print(f"   Decision: {'F1-Score (imbalanced)' if use_f1 else 'Accuracy (balanced)'}")

print("\n" + "=" * 60)
PRIMARY_METRIC = 'f1' if use_f1 else 'accuracy'
print(f"PRIMARY METRIC: {PRIMARY_METRIC.upper()}")
print("=" * 60)

### 5.2 Calculate Baseline Metrics

**Naive Baseline (Majority Class):** If we always predict the most common class, we get the majority class accuracy. This is our accuracy baseline.

**Balanced Accuracy Baseline:** For K classes, a random classifier would achieve 1/K balanced accuracy. This is more meaningful for imbalanced data.

In [ ]:
# Find the majority class
majority_class_name = counts.loc[counts['text'].idxmax(), 'sentiment']
baseline = counts['text'].max() / counts['text'].sum()

print(f"Majority class: {majority_class_name}")
print(f"Baseline accuracy (majority class): {baseline:.2f}")

In [ ]:
# Balanced accuracy baseline (random classifier)
n_classes = len(counts)
balanced_accuracy_baseline = 1.0 / n_classes

print(f"Number of classes: {n_classes}")
print(f"Balanced accuracy baseline (random): {balanced_accuracy_baseline:.2f}")

### 5.3 Create Validation Set

We split off a portion of the training data for validation. This will be used to:
- Evaluate model performance during hyperparameter tuning
- Compare models without touching the test set

In [ ]:
VALIDATION_SIZE = 0.1

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=VALIDATION_SIZE, stratify=y_train,
                                                  shuffle=True, random_state=SEED)

### 5.4 Configure Training Parameters

**Key training settings:**
- **Optimiser:** Adam - adaptive learning rate optimiser with momentum, widely used for deep learning
- **Loss:** Categorical cross-entropy - standard loss for multi-class classification
- **Training Metrics:** Accuracy, Precision, Recall, AUC (tracked by Keras during training)
- **Primary Metric:** F1-Score (macro-averaged) - computed separately after training using sklearn, since Keras doesn't provide a built-in macro-F1 metric

In [ ]:
INPUT_DIMENSION = X_train.shape[1]
OUTPUT_CLASSES = y_train.shape[1]

OPTIMIZER = 'adam'
LOSS_FUNC = 'categorical_crossentropy'

# Training metrics (tracked by Keras during training)
# Note: F1-Score (our primary metric) is computed separately using sklearn
# because Keras doesn't provide a built-in macro-averaged F1 metric
METRICS = ['categorical_accuracy',
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall'),
           # multi_label=True is about LABEL FORMAT, not task type:
           # - Our task: multi-CLASS (4 mutually exclusive classes)
           # - Our labels: one-hot encoded [[1,0,0,0], [0,1,0,0], ...]
           # - multi_label=True tells Keras to expect one-hot format
           # - multi_label=False expects integer indices [0, 1, 2, 3]
           tf.keras.metrics.AUC(name='auc', multi_label=True)]

In [ ]:
# Single-Layer Perceptron (no hidden layers)
slp_model = Sequential(name='Single_Layer_Perceptron')
slp_model.add(layers.Input(shape=(INPUT_DIMENSION,)))
slp_model.add(Dense(OUTPUT_CLASSES, activation='softmax'))
slp_model.compile(optimizer=OPTIMIZER, loss=LOSS_FUNC, metrics=METRICS)

slp_model.summary()

In [ ]:
# =============================================================================
# TRAINING CONFIGURATION
# =============================================================================

# Batch Size Selection:
# - Large datasets (>10,000 samples): Use 512 for efficient GPU utilisation
# - Small datasets (<10,000 samples): Use 32-64 for better gradient estimates
# Twitter Entity Sentiment has ~74,000 samples → Use batch size 512
BATCH_SIZE = 512

# We use DIFFERENT epoch counts for different training phases:
#
# EPOCHS_BASELINE (100): For SLP and unregularised DNN
#   - SLP converges quickly (simple model)
#   - Unregularised DNN: 100 epochs clearly shows overfitting (val_loss increasing)
#
# EPOCHS_REGULARIZED (150): For DNN with Dropout + L2
#   - WHY train longer? Regularisation SLOWS DOWN learning:
#     * Dropout randomly masks neurons, so each update uses partial information
#     * L2 penalty constrains weight updates, preventing large steps
#     * The model needs MORE iterations to reach the same level of convergence
#   - Without extra epochs, we'd stop before the model reaches its full potential
#   - With regularisation, longer training is SAFE (no overfitting risk)
#
# The trade-off: Regularisation exchanges faster convergence for overfitting protection.
# We compensate by allowing more training time.

EPOCHS_BASELINE = 100      # SLP and DNN (no regularisation)
EPOCHS_REGULARIZED = 150   # DNN with Dropout + L2 (needs more time to converge)

### 5.5 Handle Class Imbalance with Class Weights

To handle imbalanced classes, we compute **class weights** that give more importance to minority classes during training.

This makes errors on minority classes "cost more", encouraging the model to learn them better.

---

#### Why Class Weights Instead of Resampling?

| Technique | How it works | Pros | Cons |
|-----------|--------------|------|------|
| **Class Weights** | Adjusts loss function to penalise minority errors more | Simple, no data modification, works with any batch size | Doesn't add information |
| **Oversampling (SMOTE)** | Creates synthetic minority samples | Adds training data, can improve decision boundaries | Risk of overfitting to synthetic data, increases training time |
| **Undersampling** | Removes majority class samples | Balances dataset, faster training | Loses potentially useful information |

We use **class weights** because:
1. **Simplicity:** No need to modify the dataset or import additional libraries
2. **No synthetic data risk:** SMOTE can create unrealistic samples, especially for text data
3. **Efficiency:** Training time unchanged; no additional data to process
4. **Keras integration:** Native support via `class_weight` parameter in `model.fit()`

*Note: For severely imbalanced data (>10:1 ratio), combining class weights with oversampling may yield better results.*

---

#### How Class Weights Fit Our Evaluation Strategy

| Component | Role | Handles Imbalance? |
|-----------|------|-------------------|
| **Class weights** | Training technique - modifies loss function | Yes - penalises minority class errors more |
| **AUC** (tuning objective) | Evaluation metric during hyperparameter search | Yes - macro-averages across classes |
| **F1-macro** (primary metric) | Final evaluation metric | Yes - macro-averages across classes |

These three components work together consistently:
- **Class weights** help the model *learn* minority classes better (affects training loss)
- **AUC** and **F1-macro** *measure* performance across all classes equally (evaluation metrics, computed without weights)

All three treat classes fairly regardless of size, making this a coherent strategy for imbalanced data.

In [ ]:
labels = np.argmax(y_train, axis=1)
weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
CLASS_WEIGHTS = dict(enumerate(weights))

CLASS_WEIGHTS

In [ ]:
# Train the Single-Layer Perceptron
history_slp = slp_model.fit(X_train, y_train,
                            class_weight=CLASS_WEIGHTS,
                            batch_size=BATCH_SIZE, epochs=EPOCHS_BASELINE,
                            validation_data=(X_val, y_val),
                            verbose=0)
val_score_slp = slp_model.evaluate(X_val, y_val, verbose=0)[1:]

In [ ]:
# Display SLP validation metrics
print('Accuracy (Validation): {:.2f} (baseline={:.2f})'.format(val_score_slp[0], baseline))
print('Precision (Validation): {:.2f}'.format(val_score_slp[1]))
print('Recall (Validation): {:.2f}'.format(val_score_slp[2]))
print('AUC (Validation): {:.2f}'.format(val_score_slp[3]))

preds_slp_val = slp_model.predict(X_val, verbose=0).argmax(axis=1)
print('Balanced Accuracy (Validation): {:.2f} (baseline={:.2f})'.format(
    balanced_accuracy_score(y_val.argmax(axis=1), preds_slp_val), balanced_accuracy_baseline))

# Calculate F1-Score (primary metric for imbalanced data)
f1_slp_val = f1_score(y_val.argmax(axis=1), preds_slp_val, average='macro')
print(f'F1-Score (Validation): {f1_slp_val:.2f}  ← Primary Metric')

In [ ]:
def plot_training_history(history, title=None):
    """
    Plot training and validation metrics over epochs.
    Plots: (1) Loss, (2) Accuracy

    Note: We use Accuracy for training visualisation (directly tracked by Keras).
    F1-Score is computed for final evaluation since it's our primary metric.

    Parameters:
    -----------
    history : keras History object
        Training history from model.fit()
    title : str, optional
        Model name to display in plot titles (e.g., 'SLP', 'DNN')
    """
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))
    epochs = range(1, len(history.history['loss']) + 1)
    title_suffix = f' ({title})' if title else ''

    # Plot 1: Loss
    axs[0].plot(epochs, history.history['loss'], 'b-', label='Training', linewidth=1.5)
    axs[0].plot(epochs, history.history['val_loss'], 'r-', label='Validation', linewidth=1.5)
    axs[0].set_title(f'Loss{title_suffix}')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Loss')
    axs[0].legend()
    axs[0].grid(alpha=0.3)

    # Plot 2: Accuracy (directly tracked by Keras)
    axs[1].plot(epochs, history.history['categorical_accuracy'], 'b-', label='Training', linewidth=1.5)
    axs[1].plot(epochs, history.history['val_categorical_accuracy'], 'r-', label='Validation', linewidth=1.5)
    axs[1].set_title(f'Accuracy{title_suffix}')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Accuracy')
    axs[1].legend()
    axs[1].grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
# Plot SLP training history
plot_training_history(history_slp, title='SLP Baseline')

## 6. Scaling Up: Developing a Model That Overfits

The next step in the Universal ML Workflow is to build a model with **enough capacity to overfit**. If a model can't overfit, it may be too simple to learn the patterns in the data.

**Strategy:** Add hidden layers and neurons to increase model capacity.

**No regularisation applied:** We intentionally train this model **without any regularisation** (no dropout, no L2, no early stopping) to observe overfitting behaviour. In the training plots, you should see:
- Training loss continues to decrease
- Validation loss starts increasing after some epochs (overfitting)

This demonstrates why regularisation (Section 7) is necessary.

---

#### Architecture Design Decisions

**Why 64 neurons in the hidden layer?**

This is a practical starting point that balances capacity and efficiency:
- **Too few (e.g., 16):** May not have enough capacity to learn complex patterns
- **Too many (e.g., 512):** Increases overfitting risk and training time without proportional benefit
- **64 neurons:** A common choice for tabular/text data that provides sufficient capacity for most classification tasks

**Why only 1 hidden layer instead of 2-3?**

Per the **Universal ML Workflow**, the goal of this step is to demonstrate that the model *can* overfit—proving it has sufficient capacity to capture the underlying patterns. Once overfitting is observed:

1. **Capacity is proven sufficient:** If the model overfits, it can learn the training data's complexity
2. **No need for more depth:** Adding layers would increase overfitting further without benefit
3. **Regularise, don't expand:** The next step (Section 7) is to *reduce* overfitting through regularisation, not to add more capacity

If this 1-layer model *couldn't* overfit (training and validation loss both plateau high), we would then add more layers. But since it does overfit, the architecture is adequate.

*"The right question is not 'How many layers?' but 'Can it overfit?' If yes, regularise. If no, add capacity."*

### 6.1 Build a Deep Neural Network (DNN)

Let's add a hidden layer with 64 neurons and ReLU activation:

In [ ]:
# Deep Neural Network (1 hidden layer, no dropout for overfitting demo)
dnn_model = Sequential(name='Deep_Neural_Network')
dnn_model.add(layers.Input(shape=(INPUT_DIMENSION,)))
dnn_model.add(Dense(64, activation='relu'))
dnn_model.add(Dense(OUTPUT_CLASSES, activation='softmax'))
dnn_model.compile(optimizer=OPTIMIZER, loss=LOSS_FUNC, metrics=METRICS)

dnn_model.summary()

In [ ]:
# Train the Deep Neural Network (without regularisation to demonstrate overfitting)
# Using EPOCHS_BASELINE: 100 epochs is enough to clearly show overfitting behaviour
history_dnn = dnn_model.fit(X_train, y_train,
                            class_weight=CLASS_WEIGHTS,
                            batch_size=BATCH_SIZE, epochs=EPOCHS_BASELINE,
                            validation_data=(X_val, y_val),
                            verbose=0)
val_score_dnn = dnn_model.evaluate(X_val, y_val, verbose=0)[1:]

In [ ]:
# Plot DNN training history (expect overfitting: val_loss increasing)
plot_training_history(history_dnn, title='DNN - No Regularisation')

In [ ]:
# Display DNN validation metrics
print('Accuracy (Validation): {:.2f} (baseline={:.2f})'.format(val_score_dnn[0], baseline))
print('Precision (Validation): {:.2f}'.format(val_score_dnn[1]))
print('Recall (Validation): {:.2f}'.format(val_score_dnn[2]))
print('AUC (Validation): {:.2f}'.format(val_score_dnn[3]))

preds_dnn_val = dnn_model.predict(X_val, verbose=0).argmax(axis=1)
print('Balanced Accuracy (Validation): {:.2f} (baseline={:.2f})'.format(
    balanced_accuracy_score(y_val.argmax(axis=1), preds_dnn_val), balanced_accuracy_baseline))

# Calculate F1-Score (primary metric for imbalanced data)
f1_dnn_val = f1_score(y_val.argmax(axis=1), preds_dnn_val, average='macro')
print(f'F1-Score (Validation): {f1_dnn_val:.2f}  ← Primary Metric')

## 7. Regularising Your Model and Tuning Hyperparameters

Now we address the overfitting observed in Section 6 by adding **regularisation**. We use two complementary techniques:

| Technique | How it works | Effect |
|-----------|--------------|--------|
| **Dropout** | Randomly drops neurons during training | Acts like ensemble averaging, reduces co-adaptation |
| **L2 (Weight Decay)** | Adds penalty for large weights to loss | Keeps weights small, smoother decision boundaries |

**Same architecture, different regularisation:** We keep the same 1-layer architecture (64 neurons) as Section 6, so the only difference is regularisation. This allows a fair comparison.

Using **Hyperband** for efficient hyperparameter tuning to find optimal regularisation strengths.

---

#### Why Hyperband?

| Method | How it works | Pros | Cons |
|--------|--------------|------|------|
| **Grid Search** | Tries all combinations exhaustively | Thorough, reproducible | Exponentially expensive, wastes time on bad configs |
| **Random Search** | Samples random combinations | More efficient than grid, good coverage | Still trains all configs to completion |
| **Bayesian Optimisation** | Models performance to guide search | Sample-efficient, finds good configs fast | Complex to implement, overhead per iteration |
| **Hyperband** | Early stopping of poor performers | Very efficient for deep learning, parallelisable | May discard slow starters prematurely |

We use **Hyperband** because:
1. **Efficiency:** Eliminates poor configurations early, focusing resources on promising ones
2. **Deep learning fit:** Training epochs are a natural "resource" to allocate adaptively
3. **Keras Tuner integration:** Native support via `kt.Hyperband`
4. **Scalability:** Handles many hyperparameters efficiently

**How Hyperband works:**
1. Start training many configurations for a few epochs
2. Evaluate and keep the top performers
3. Train survivors for more epochs
4. Repeat until only the best remain

### 7.1 Hyperband Search

In [ ]:
# Hyperband Model Builder for Multi-Class Twitter Entity Sentiment Classification
def build_model_hyperband(hp):
    """
    Build Twitter Entity Sentiment model with FIXED architecture (1 hidden layer, 64 neurons).
    Same architecture as Section 6 DNN - only tunes regularisation and learning rate.
    Uses Adam optimiser (same as Section 6) for fair comparison.
    """
    model = keras.Sequential()
    model.add(layers.Input(shape=(INPUT_DIMENSION,)))

    # L2 regularisation strength
    l2_reg = hp.Float('l2_reg', 1e-5, 1e-2, sampling='log')

    # Fixed architecture: 1 hidden layer with 64 neurons (same as Section 6)
    model.add(layers.Dense(64, activation='relu',
                           kernel_regularizer=regularizers.l2(l2_reg)))
    dropout_rate = hp.Float('dropout', 0.0, 0.5, step=0.1)
    model.add(layers.Dropout(dropout_rate))

    # Output layer for multi-class classification
    model.add(layers.Dense(OUTPUT_CLASSES, activation='softmax'))

    lr = hp.Float('lr', 1e-4, 1e-2, sampling='log')
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss=LOSS_FUNC,
        metrics=METRICS
    )
    return model

In [ ]:
# Configure Hyperband tuner
# ===========================================================================
# TUNING OBJECTIVE: Why AUC instead of F1?
# ===========================================================================
# We use F1-Score as our PRIMARY METRIC for final evaluation, but AUC for tuning:
#
# 1. Keras limitation: No built-in macro-averaged F1 metric available during training
#    - F1 requires computing precision/recall across all classes, then averaging
#    - Keras metrics are computed per-batch, making macro-F1 impractical
#
# 2. AUC is a better tuning objective:
#    - AUC is threshold-independent (evaluates across all decision thresholds)
#    - F1 is threshold-dependent (requires choosing a cutoff for predictions)
#    - Both penalise models that ignore minority classes
#    - Models with high AUC tend to achieve high F1 at optimal thresholds
#
# 3. Alternative approaches (not used here):
#    - Custom Keras metric (complex, slower)
#    - Validation callback computing F1 each epoch (adds overhead)
#
# Final evaluation uses sklearn's f1_score(average='macro') as primary metric.
# ===========================================================================

TUNING_OBJECTIVE = 'val_categorical_accuracy' if PRIMARY_METRIC == 'accuracy' else 'val_auc'

tuner = kt.Hyperband(
    build_model_hyperband,
    objective=TUNING_OBJECTIVE,
    max_epochs=20,
    factor=3,
    directory='twitter_entity_hyperband',
    project_name='twitter_entity_tuning',
    overwrite=True  # Ensure fresh tuning on each run
)

print(f"Tuning objective: {TUNING_OBJECTIVE}")
print("(Note: Final evaluation uses F1-Score as primary metric)")

# Run Hyperband search
tuner.search(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=BATCH_SIZE,
    class_weight=CLASS_WEIGHTS
)

In [ ]:
# Get best hyperparameters
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters found by Hyperband:")
print(f"  L2 Regularisation: {best_hp.get('l2_reg'):.6f}")
print(f"  Dropout Rate: {best_hp.get('dropout')}")
print(f"  Learning Rate: {best_hp.get('lr'):.6f}")

### 7.2 Sanity Check and Final Retraining

After finding the best hyperparameters, we follow a two-step process:

1. **Sanity Check:** Retrain with the best hyperparameters using training and validation data to visually confirm the model is not overfitting. This validates that Hyperband found hyperparameters that generalise well.

2. **Final Refit:** Combine training and validation sets and retrain without validation. Since the hyperparameters have been validated, we maximise the data available for the final model.

---

#### Why This Two-Step Approach?

| Step | Purpose | Validation Data |
|------|---------|-----------------|
| **Sanity Check** | Confirm hyperparameters prevent overfitting | ✓ Used for monitoring |
| **Final Refit** | Maximise training data for production model | ✗ Merged into training |

Once the sanity check confirms no overfitting, we can confidently combine all available data for the final model.

In [ ]:
# =============================================================================
# STEP 1: SANITY CHECK - Retrain with validation to confirm no overfitting
# =============================================================================

# Extract the number of epochs from the best trial
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]
best_epochs = best_trial.best_step + 1  # best_step is 0-indexed

print("=" * 60)
print("SANITY CHECK: Retraining with Validation")
print("=" * 60)
print(f"Training for {best_epochs} epochs (matched from Hyperband's best trial)")
print("Purpose: Visually confirm the hyperparameters prevent overfitting\n")

# Build a fresh model with the best hyperparameters
sanity_model = tuner.hypermodel.build(best_hp)

history_sanity = sanity_model.fit(
    X_train, y_train,
    epochs=best_epochs,
    batch_size=BATCH_SIZE,
    class_weight=CLASS_WEIGHTS,
    validation_data=(X_val, y_val),  # Include validation for monitoring
    verbose=0
)

print("Sanity check training complete.")

In [ ]:
# Plot sanity check training history (with validation curves)
plot_training_history(history_sanity, title=f'Sanity Check - Best Hyperparameters ({best_epochs} epochs)')

# Verify no overfitting: validation loss should not increase significantly
val_losses = history_sanity.history['val_loss']
min_val_loss_epoch = val_losses.index(min(val_losses)) + 1
final_val_loss = val_losses[-1]
min_val_loss = min(val_losses)

print(f"\nSanity Check Results:")
print(f"  Minimum validation loss: {min_val_loss:.4f} at epoch {min_val_loss_epoch}")
print(f"  Final validation loss: {final_val_loss:.4f}")
if final_val_loss <= min_val_loss * 1.1:  # Within 10% of minimum
    print("  ✓ No significant overfitting detected - hyperparameters are validated")
else:
    print("  ⚠ Some overfitting detected - consider adjusting epochs")

In [ ]:
# =============================================================================
# STEP 2: FINAL REFIT - Combine data and retrain for production
# =============================================================================

# Combine training and validation sets for final model
X_train_full = np.vstack([X_train, X_val])
y_train_full = np.vstack([y_train, y_val])

print("=" * 60)
print("FINAL REFIT: Training on Combined Data")
print("=" * 60)
print(f"Training data: {X_train.shape[0]:,} samples")
print(f"Validation data: {X_val.shape[0]:,} samples")
print(f"Combined data: {X_train_full.shape[0]:,} samples")
print(f"  → {(X_train_full.shape[0] / X_train.shape[0] - 1) * 100:.1f}% more training data")

# Recompute class weights on combined data
labels_full = np.argmax(y_train_full, axis=1)
weights_full = compute_class_weight('balanced', classes=np.unique(labels_full), y=labels_full)
CLASS_WEIGHTS_FULL = dict(enumerate(weights_full))
print(f"\nClass weights (combined): {CLASS_WEIGHTS_FULL}")

# Build and train final model on combined data
print(f"\nRetraining for {best_epochs} epochs on combined data...")

opt_model = tuner.hypermodel.build(best_hp)

opt_model.fit(
    X_train_full, y_train_full,
    epochs=best_epochs,
    batch_size=BATCH_SIZE,
    class_weight=CLASS_WEIGHTS_FULL,
    verbose=0
    # No validation_data - merged into training
    # No plotting needed - sanity check already validated the hyperparameters
)

print("\n✓ Final model training complete on combined dataset.")

### 7.3 Final Model Evaluation on Test Set

Now we evaluate our best model on the held-out test set that was never used during training or tuning.

In [ ]:
# Final evaluation on test set
test_score = opt_model.evaluate(X_test, y_test, verbose=0)[1:]
preds_test = opt_model.predict(X_test, verbose=0).argmax(axis=1)

# Calculate F1-Score (our primary metric)
test_f1 = f1_score(y_test.argmax(axis=1), preds_test, average='macro')

print('=' * 50)
print('FINAL TEST SET RESULTS')
print('=' * 50)
print(f'F1-Score (Test): {test_f1:.4f}  ← Primary Metric')
print(f'Accuracy (Test): {test_score[0]:.4f} (baseline={baseline:.4f})')
print(f'Precision (Test): {test_score[1]:.4f}')
print(f'Recall (Test): {test_score[2]:.4f}')
print(f'AUC (Test): {test_score[3]:.4f}')
print(f'Balanced Accuracy (Test): {balanced_accuracy_score(y_test.argmax(axis=1), preds_test):.4f}')

In [ ]:
# Display confusion matrix for test predictions
fig, ax = plt.subplots(figsize=(8, 6))
cm = confusion_matrix(y_test.argmax(axis=1), preds_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
disp.plot(ax=ax, cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Test Set Predictions')
plt.tight_layout()
plt.show()

# Print per-class recall (what % of each class was correctly identified)
print("\nPer-Class Recall:")
for i, class_name in enumerate(label_encoder.classes_):
    class_mask = y_test.argmax(axis=1) == i
    class_recall = (preds_test[class_mask] == i).mean()
    print(f"  {class_name}: {class_recall:.2%} ({class_mask.sum()} samples)")

---

## 8. Results Summary

The following dynamically-generated table compares all models trained in this notebook.

In [ ]:
# =============================================================================
# RESULTS SUMMARY
# =============================================================================

# Create results DataFrame
# Note: Final model (DNN Dropout + L2) shows only test metrics
# because validation data was combined with training for final model
results = pd.DataFrame({
    'Model': ['Naive Baseline', 'SLP (No Hidden)', 'DNN (No Regularisation)', 'DNN (Dropout + L2)'],
    'Accuracy': [baseline, val_score_slp[0], val_score_dnn[0], test_score[0]],
    'F1-Score': [0.0, f1_slp_val, f1_dnn_val, test_f1],
    'Dataset': ['N/A', 'Validation', 'Validation', 'Test']
})

print("=" * 70)
print("MODEL COMPARISON - RESULTS SUMMARY")
print("=" * 70)
print(f"Primary Metric: F1-SCORE (imbalance ratio: {imbalance_ratio:.2f}:1)")
print("=" * 70)
print(results.to_string(index=False, float_format='{:.4f}'.format))
print("=" * 70)
print(f"\nKey Observations:")
print(f"  - All models outperform naive baseline ({baseline:.2%} accuracy)")
print(f"  - DNN with Dropout + L2 trained on combined train+val data")
print(f"  - Final test F1-Score: {test_f1:.4f}")

---

## 9. Key Takeaways

### Decision Framework Summary

| Decision | Threshold | This Dataset | Choice | Reference |
|----------|-----------|--------------|--------|----------|
| **Hold-Out vs K-Fold** | > 10,000 samples | ~74,000 samples | Hold-Out | Kohavi (1995); Chollet (2021) |
| **Accuracy vs F1-Score** | > 3:1 imbalance | ~1.7:1 ratio | F1-Score* | He and Garcia (2009) |

*Note: The imbalance ratio for this dataset is mild (~1.7:1), but we still use F1-Score for consistency with the workflow and to demonstrate the data-driven decision process. The code dynamically selects the metric based on the computed ratio.*

### Lessons Learned

1. **Data-Driven Metric Selection:** We compute the imbalance ratio from data and use F1-Score if > 3:1 to ensure fair evaluation across all classes.

2. **Data-Driven Evaluation Protocol:** With > 10,000 samples and deep learning, hold-out validation provides reliable estimates while being computationally efficient.

3. **Class Imbalance Handling:** Using class weights during training improves performance on minority classes. This is simpler than resampling techniques (SMOTE, undersampling) and avoids synthetic data risks.

4. **Simple Models Can Work Well:** The SLP achieved competitive F1-Score with good feature engineering (TF-IDF).

5. **Regularisation Prevents Overfitting:** The unregularised DNN showed overfitting; combining **Dropout + L2 regularisation** controls this effectively—no need for deeper architectures once overfitting is demonstrated.

6. **Maximise Data for Final Model:** After hyperparameter tuning, we combine training and validation sets for the final model. The validation set has served its purpose (model selection), and more training data leads to better generalisation.

7. **Technique Scope:** We use only techniques from Chapters 1–4 of *Deep Learning with Python* (Chollet, 2021). Early stopping and word embeddings, while useful, are outside this scope.

8. **4-Class vs 3-Class Sentiment:** This dataset includes an "Irrelevant" class, which is common in real-world entity sentiment analysis where not all mentions express sentiment.

9. **Code Reusability:** The core workflow and code patterns are identical to the Twitter US Airline Sentiment notebook—only the data loading and class-specific details change. This demonstrates that the Universal ML Workflow is truly universal.

### References

- Chollet, F. (2021) *Deep learning with Python*. 2nd edn. Shelter Island, NY: Manning Publications.

- Haddi, E., Liu, X. and Shi, Y. (2013) 'The role of text pre-processing in sentiment analysis', *Procedia Computer Science*, 17, pp. 26–32.

- Hastie, T., Tibshirani, R. and Friedman, J. (2009) *The elements of statistical learning: data mining, inference, and prediction*. 2nd edn. New York: Springer.

- He, H. and Garcia, E.A. (2009) 'Learning from imbalanced data', *IEEE Transactions on Knowledge and Data Engineering*, 21(9), pp. 1263–1284.

- Kohavi, R. (1995) 'A study of cross-validation and bootstrap for accuracy estimation and model selection', *IJCAI*, 2, pp. 1137–1145.

---

## Appendix: Modular Helper Functions

For cleaner code organisation, you can wrap the model building and training patterns into reusable functions.

In [ ]:
# =============================================================================
# MODULAR HELPER FUNCTIONS
# =============================================================================

def build_multiclass_nlp_classifier(input_dim, num_classes, hidden_units=None,
                                     dropout=0.0, l2_reg=0.0,
                                     optimizer='adam', learning_rate=None, name=None):
    """
    Build a multi-class NLP classification neural network.

    Parameters:
    -----------
    input_dim : int
        Number of input features (TF-IDF vector dimension)
    num_classes : int
        Number of output classes
    hidden_units : list of int, optional
        Neurons per hidden layer, e.g., [64] or [128, 64]
    dropout : float
        Dropout rate (0.0 to 0.5)
    l2_reg : float
        L2 regularisation strength
    learning_rate : float, optional
        Custom learning rate
    name : str, optional
        Model name

    Returns:
    --------
    keras.Sequential : Compiled model ready for training
    """
    model = Sequential(name=name)
    model.add(layers.Input(shape=(input_dim,)))

    hidden_units = hidden_units or []
    kernel_reg = regularizers.l2(l2_reg) if l2_reg > 0 else None

    for units in hidden_units:
        model.add(Dense(units, activation='relu', kernel_regularizer=kernel_reg))
        if dropout > 0:
            model.add(Dropout(dropout))

    model.add(Dense(num_classes, activation='softmax'))

    if learning_rate is not None:
        opt = keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        opt = optimizer

    metrics = ['categorical_accuracy',
               tf.keras.metrics.Precision(name='precision'),
               tf.keras.metrics.Recall(name='recall'),
               tf.keras.metrics.AUC(name='auc', multi_label=True)]

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=metrics)
    return model


def train_with_class_weights(model, X_train, y_train, X_val, y_val,
                              batch_size=512, epochs=100, verbose=0):
    """Train model with automatic class weight computation."""
    labels = np.argmax(y_train, axis=1)
    weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
    class_weights = dict(enumerate(weights))

    return model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=batch_size,
        epochs=epochs,
        class_weight=class_weights,
        verbose=verbose
    )


def evaluate_multiclass_nlp(model, X, y_true_onehot, average='macro'):
    """
    Evaluate multi-class NLP classification model.

    Returns:
    --------
    dict : Dictionary containing all multi-class classification metrics
    """
    y_pred_proba = model.predict(X, verbose=0)
    y_pred = y_pred_proba.argmax(axis=1)
    y_true = y_true_onehot.argmax(axis=1)

    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred, average=average),
        'balanced_accuracy': balanced_accuracy_score(y_true, y_pred),
    }

    return metrics


# =============================================================================
# USAGE EXAMPLES
# =============================================================================
#
# # Build models
# slp = build_multiclass_nlp_classifier(INPUT_DIMENSION, OUTPUT_CLASSES, name='SLP')
# mlp = build_multiclass_nlp_classifier(INPUT_DIMENSION, OUTPUT_CLASSES,
#                                       hidden_units=[64], name='MLP')
# mlp_reg = build_multiclass_nlp_classifier(INPUT_DIMENSION, OUTPUT_CLASSES,
#                                           hidden_units=[64], dropout=0.3, l2_reg=0.001,
#                                           learning_rate=0.001)
#
# # Train with class weights
# history = train_with_class_weights(mlp, X_train, y_train, X_val, y_val)
#
# # Evaluate
# metrics = evaluate_multiclass_nlp(mlp, X_val, y_val)
# print(f"Accuracy: {metrics['accuracy']:.4f}, F1: {metrics['f1']:.4f}")